## Continuous-time Quantum Monte Carlo of a Single Spin

The Hamiltonian of a single spin in a magnetic field is
$$
H = -\\frac{h}{2} \\sigma_z - \\frac{\\Gamma}{2} \\sigma_x.
$$

### Monte Carlo Implementation

The analytical solution for the magnetization $\\langle \\sigma_x \\rangle= \\frac{2}{\\beta}\\partial_\\Gamma \\log Z$ is
$$
\\langle \\sigma_x \\rangle = \\frac{\\Gamma}{\\sqrt{\\Gamma^2+h^2}} \\tanh\\left( \\frac{\\beta}{2} \\sqrt{h^2+\\Gamma^2} \\right).
$$
We map the quantum system to a classical continuous 1D spin chain of length $\\beta$. The total energy of the system is
$$
E_\\mathrm{tot} = -\\frac{h}{2} S_z^\\mathrm{tot}- \\frac{2n}{\\beta}\\log \\left(\\frac{d\\tau \\Gamma}{2}\\right),
$$
where $2n$ is the number of domain walls and $\\sigma(\\tau)$ is the magnetization, and $S_z^\\mathrm{tot}=\\frac{1}{\\beta}\\int_0^\\beta d\\tau \\sigma(\\tau)$ the total magnetization. From the above result, we can calculate the acceptance probabilities.

In [ ]:
# Import Required Libraries
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import copy
from sys import stdout
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9

# Only needed for Mac OS users
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [ ]:
# Define Analytical Solution
sx = lambda gamma, h, beta: gamma / np.sqrt(gamma**2 + h**2) * np.tanh(beta * np.sqrt(h**2 + gamma**2) / 2)

In [ ]:
# Worldline Class Implementation
class Worldline:

    def __init__(self, beta, gamma, h=0):
        self.beta = beta     # inverse temperature \\beta
        self.gamma = gamma   # transverse field \\Gamma
        self.h = h           # magnetic field h
        self.s0 = 1          # spin at \\tau=0 (whether the first spin is up or down)
        self.kinks = []      # position of kinks (domain walls), sorted from 0 to beta
        self.magnobs = []    # observations (measurements) of the magnetization

    def sz(self, kinks, s0):
        # Compute the magnetization of a given configuration (i.e., given first spin and domain walls)
        if len(kinks) > 0:
            sz = s0 * (kinks[0] + (self.beta - kinks[-1]))
            for i in range(len(kinks) - 1):
                sz += s0 * (-1)**(i + 1) * (kinks[i + 1] - kinks[i])
            sz /= self.beta
        else:
            sz = s0 * self.beta
        return sz

    def szdifference_add(self, ta, tb):
        # Compute the difference in magnetization when adding two domain walls
        new_kinks = copy.copy(self.kinks)
        new_kinks += (ta, tb)
        new_kinks.sort()
        new_s0 = -1 * self.s0 if ta > tb else self.s0
        return self.sz(new_kinks, new_s0) - self.sz(self.kinks, self.s0)

    def szdifference_rem(self, i, j):
        # Compute the difference in magnetization when removing two domain walls
        new_kinks = copy.copy(self.kinks)
        if i > j:
            del new_kinks[i]
            del new_kinks[j]
            new_s0 = -1 * self.s0
        else:
            del new_kinks[j]
            del new_kinks[i]
            new_s0 = self.s0
        return self.sz(new_kinks, new_s0) - self.sz(self.kinks, self.s0)

    def insertUpdate(self):
        # Insert two domain walls if accepted
        ta, tb = rnd.uniform(0, self.beta), rnd.uniform(0, self.beta)
        n = len(self.kinks)
        ratio = np.exp(self.beta * 0.5 * self.h * self.szdifference_add(ta, tb)) if self.h != 0 else 1
        p_acc = ratio * (self.beta * self.gamma / 2)**2 / ((n + 1) * (n + 2))
        if rnd.rand() < p_acc:
            self.kinks += (ta, tb)
            self.kinks.sort()
            if ta > tb:
                self.s0 *= -1

    def removeUpdate(self):
        # Remove two domain walls if accepted
        if len(self.kinks) < 2:
            return
        i, j = rnd.randint(0, len(self.kinks)), rnd.randint(0, len(self.kinks))
        while i == j:
            j = rnd.randint(0, len(self.kinks))
        n = len(self.kinks)
        ratio = np.exp(self.beta * 0.5 * self.h * self.szdifference_rem(i, j)) if self.h != 0 else 1
        p_acc = ratio * n * (n - 1) / (self.gamma * self.beta / 2)**2
        if rnd.rand() < p_acc:
            if i > j:
                del self.kinks[i]
                del self.kinks[j]
                self.s0 *= -1
            else:
                del self.kinks[j]
                del self.kinks[i]

    def update(self):
        # Perform a Monte Carlo update
        if rnd.rand() < 0.5:
            self.insertUpdate()
        else:
            self.removeUpdate()

    def measure(self):
        # Measure the magnetization of the current configuration
        self.magnobs.append(2 * len(self.kinks) / (self.beta * self.gamma))

In [ ]:
# Binning Analysis Function
def binning_analysis(samples, n_levels):
    bins = np.array(samples)
    errors = np.zeros(n_levels + 1)
    errors[0] = np.std(bins) / np.sqrt(len(bins) - 1)
    for k in range(n_levels):
        bins = np.array([(bins[2 * i] + bins[2 * i + 1]) / 2. for i in range(len(bins) // 2)])
        errors[k + 1] = np.std(bins) / np.sqrt(len(bins) - 1)
    tau = 0.5 * (errors[-1]**2 / np.std(samples)**2 * (len(samples) - 1.) - 1.)
    return errors, tau

In [ ]:
# Monte Carlo Simulation
steps = int(2**17)           # number of Monte Carlo steps
n_levels = 9                 # number of binning levels
thermsteps = steps // 5      # number of thermalization steps
beta = 1.                    # inverse temperature \\beta
h = 1.                       # magnetic field h
results = []

for gamma in np.linspace(0.05, 6., 100):
    wl = Worldline(beta, gamma, h)
    for _ in range(thermsteps):
        wl.update()
    for _ in range(steps):
        wl.update()
        wl.measure()
    mean = np.mean(wl.magnobs)
    errors, tau = binning_analysis(wl.magnobs, n_levels)
    print(f"Gamma = {gamma:.5f}, <sigma_x> = {mean:.5f} ± {errors[-1]:.5f}, tau = {tau:.5f}")
    results.append([gamma, mean, errors[-1], tau])

results = np.transpose(np.array(results))

In [ ]:
# Visualization of Results
plt.figure(figsize=(10, 7))
plt.errorbar(results[0], results[1], results[2], fmt='.', label='Monte Carlo', capsize=4)
plt.plot(results[0], sx(results[0], h, beta), label='Exact')
plt.xlabel('$\\Gamma$')
plt.ylabel('$\\langle \\sigma_x \\rangle$')
plt.title(f'Ising Spin in Transverse Field ($\\beta={beta}$)')
plt.legend(loc='best')
plt.show()